In [1]:
import torch
import numpy as np

# relative imports
from cnn import CNN
from transpose_cnn import TransposeCNN

# constants and hyperparameters
LEARN_RATE = 1e-4
BATCH_SIZE = 64
NUM_TRAIN = 60000
NUM_TEST = 10000
NUM_WORKERS = 4
NUM_CHAN =1
NUM_EPOCHS = 20
Z_DIM = 128

# try to get gpu device, if not just use cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print('[INFO] using \'{}\' device'.format(device))

[INFO] using 'cpu' device


In [2]:
# initialize critic (CNN) model
critic = CNN(
    in_chan=NUM_CHAN, 
    out_dim=1, 
    hid_act=torch.nn.ReLU(), 
    out_act=torch.nn.Identity(), 
    layer_norm=False
)

# initialize generator (TransposeCNN) model
generator = TransposeCNN(
    in_dim=Z_DIM,
    out_chan=NUM_CHAN,
    hid_act=torch.nn.ReLU(),
    out_act=torch.nn.Tanh(),
    layer_norm=True
)

print('[INFO] critic structure \n{}'.format(critic))
print('[INFO] generator structure \n{}'.format(generator))

[INFO] critic structure 
CNN(
  (hid_act): ReLU()
  (out_act): Identity()
  (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv_4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv_5): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm_1): Identity()
  (norm_2): Identity()
  (norm_3): Identity()
  (norm_4): Identity()
  (norm_5): Identity()
  (fc_1): Linear(in_features=2048, out_features=1, bias=True)
)
[INFO] generator structure 
TransposeCNN(
  (hid_act): ReLU()
  (out_act): Tanh()
  (fc_1): Linear(in_features=128, out_features=2048, bias=True)
  (conv_1): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv_2): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv_3): ConvTranspose2d(1